<a href="https://colab.research.google.com/github/Jinyoung3/AI-projects/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchtext
from torchtext.data import get_tokenizer
import tokenize
import io
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem import PorterStemmer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib inline

In [ ]:
!wget "https://drive.google.com/u/0/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download" -O english_python_data.txt

--2023-11-14 01:30:10--  https://drive.google.com/u/0/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download
Resolving drive.google.com (drive.google.com)... 142.251.2.102, 142.251.2.113, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download [following]
--2023-11-14 01:30:10--  https://drive.google.com/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/phhg8q84cdiel00et4l9kfsjf8q0n3s4/1699925400000/02008525212197398114/*/1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO?e=download&uuid=ba0d505c-ad72-48f1-ae0f-571e9c7eacec [following]
--2023-11-14 01:30:11--  https://doc-14-3o-docs.googleusercontent.com/do

In [ ]:
with open('english_python_data.txt',"r") as data_file:
  print(data_file.readlines()[:5]) # Printing out the first 5 lines of the data

['# write a python program to add two numbers \n', 'num1 = 1.5\n', 'num2 = 6.3\n', 'sum = num1 + num2\n', "print(f'Sum: {sum}')\n"]


In [ ]:
with open('english_python_data.txt',"r") as data_file:
  data_lines = data_file.readlines()
  dps = []
  dp = None
  for line in data_lines:
    if line[0] == "#":
      if dp:
        dp['solution'] = ''.join(dp['solution'])
        dps.append(dp)
      dp = {"question": None, "solution": []}
      dp['question'] = line[1:]
    else:
      dp["solution"].append(line)

# converting the data to a table for easier viewing
dataset = pd.DataFrame(dps)
dataset

,question,solution
0,write a python program to add two numbers \n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,write a python function to add two user provi...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,write a program to find and print the largest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...
3,write a program to find and print the smalles...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,Write a python function to merge two given li...,"def merge_lists(l1, l2):\n return l1 + l2\n..."
...,...,...
4952,Write a program to print bit wise AND of two ...,a = 60 # 60 = 0011 1100\nb = 13 ...
4953,Write a program to print bit wise OR of two n...,"a = 60\nb = 13\n\nc = a | b\nprint(""OR"", c)\n\n\n"
4954,Write a program to print bit wise XOR of two ...,"a = 60\nb = 13\n\nc = a ^ b\nprint(""XOR"", c)\n..."
4955,Write a program to calculate Binary Ones Comp...,"a = 60\n\nc = ~a\nprint(""Binary Ones Complemen..."


In [ ]:
print(dataset.loc[0,'question'])#location is loc  and grabs the first element from question and solution
print(dataset.loc[0,'solution'])

 write a python program to add two numbers 

num1 = 1.5
num2 = 6.3
sum = num1 + num2
print(f'Sum: {sum}')





In [ ]:
SOS_token = 1
EOS_token = 2#creating a class that can hold python and english, creates vocabulary and holds vocab for both languages for encoding and decoding

class Language:
  def __init__(self,name):
    self.name = name
    self.word2index = {'SOS':1, 'EOS':2}
    self.word2count = {}
    self.index2word = {1:'SOS',2:'EOS'}
    self.n_words = 3

  def addSentence(self, sentence):
    for word in sentence:
      self.addWord(word)
  def addWord(self, word):
    if word not in self.word2index.keys():
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words +=1
    else:
      self.word2count[word]+=1

In [ ]:
indicies_to_ignore = []
tokenized_python = []#splitting into smaller lines or characters
for i in range(dataset.shape[0]):
  try:
    solution = dataset.loc[i,'solution'].strip("\n ")
    tokenized_code = [token.string for token in list(tokenize.generate_tokens(io.StringIO(solution).readline)) if token.string]
    tokenized_python.append(tokenized_code)
  except:
    indicies_to_ignore.append(i)
print(f'Total Acceptable Examples: {len(tokenized_python)}')

Total Acceptable Examples: 4928


In [ ]:
dataset_copy = dataset.copy()#making a copy so i can modify the copied version instead of the original value
dataset_copy.drop(indicies_to_ignore,inplace = True)
dataset_copy.reset_index(drop=True, inplace=True)
dataset_copy

,question,solution
0,write a python program to add two numbers \n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,write a python function to add two user provi...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,write a program to find and print the largest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...
3,write a program to find and print the smalles...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,Write a python function to merge two given li...,"def merge_lists(l1, l2):\n return l1 + l2\n..."
...,...,...
4923,Write a program to print bit wise AND of two ...,a = 60 # 60 = 0011 1100\nb = 13 ...
4924,Write a program to print bit wise OR of two n...,"a = 60\nb = 13\n\nc = a | b\nprint(""OR"", c)\n\n\n"
4925,Write a program to print bit wise XOR of two ...,"a = 60\nb = 13\n\nc = a ^ b\nprint(""XOR"", c)\n..."
4926,Write a program to calculate Binary Ones Comp...,"a = 60\n\nc = ~a\nprint(""Binary Ones Complemen..."


In [ ]:
cleaned_english = []#stemming is cutting off words, so running will be in the word run cutting off ing, and runs will be run so get rid of s, using such rules
url_pattern = re.compile(r"https?://\S+")
tokenizer = get_tokenizer('basic_english')
stemmer = PorterStemmer()#lemmatization so runs an algorithm to smartly get rid of such characters, ponies and stemming will be pon which is bad so lemmatization will cut off the
#influential words so it cuts off and makes sense so walks and walking will be walk and walk but stemming will make it walk and wal get rid of last few characters with no reason so use lemma to smartly make it a word that makes sense

for i in range(dataset_copy.shape[0]):
  sentence = dataset_copy.loc[i,'question']
  sentence = sentence.lower()
  sentence = sentence.strip("\n")
  sentence = url_pattern.sub('', sentence)
  sentence = re.sub(r"([.!?])","",sentence)
  sentence = re.sub(r'^\d+ ',"", sentence)#d means digit and + means at least one, you want to throw it out because you are only inputting english words and theseare not english
  sentence = tokenizer(sentence)
  sentence = [stemmer.stem(word) for word in sentence]
  cleaned_english.append(sentence)
print(f'Number of Sentences: {len(cleaned_english)}')

Number of Sentences: 4928


In [ ]:
prepared_data = list(zip(cleaned_english,tokenized_python))

In [ ]:
english_lang = Language('english')#builds two vocabularies
python_lang = Language('python')#populates the vocabularies
for(english, python) in prepared_data:
  english_lang.addSentence(english)
  python_lang.addSentence(python)

In [ ]:
print(f'Number of words in the English Vocabulary: {english_lang.n_words}')
print(f'Number of words in the Python Vocabulary: {python_lang.n_words}')


Number of words in the English Vocabulary: 1987
Number of words in the Python Vocabulary: 6873


In [ ]:
max_length_eng = 0
max_length_python = 0
avg_eng_length = 0
avg_python_length = 0
number_of_pairs = 0
for(english,python) in prepared_data:
  if len(english)>max_length_eng:
    max_length_eng = len(english)
  if len(python)>max_length_python:
    max_length_python = len(python)

  number_of_pairs +=1#one english word and corresponding python code
  avg_eng_length += len(english)
  avg_python_length += len(python)

print(f'Max length of an english sentence: {max_length_eng}')
print(f'Max length of a python code: {max_length_python}')

print(f'avereage length of an english sentence: {avg_eng_length/number_of_pairs}')
print(f'avereage length of a python code: {avg_python_length/number_of_pairs}')

Max length of an english sentence: 71
Max length of a python code: 750
avereage length of an english sentence: 12.633928571428571
avereage length of a python code: 44.77617694805195


In [ ]:
final_prepared_data = []
for (english, python) in prepared_data:
  english_sentence = ['SOS']
  english_sentence.extend(english)
  english_sentence.append('EOS')

  python_sentence = ['SOS']
  python_sentence.extend(python)
  python_sentence.append('EOS')

  tokenized_english = []
  for word in english_sentence:
    tokenized_english.append(english_lang.word2index[word])

  tokenized_python = []
  for word in python_sentence:
    tokenized_python.append(python_lang.word2index[word])

  padding_needed_eng = 11- len(tokenized_english)
  padding_needed_python = 21 - len(tokenized_python)

  if padding_needed_eng <=0:
    tokenized_english = tokenized_english[:20]
    tokenized_english.append(EOS_token)
    tokenized_english = torch.from_numpy(np.array(tokenized_english))
  else:
    tokenized_english = nn.functional.pad(torch.from_numpy(np.array(tokenized_english)), (0,padding_needed_eng))

  if padding_needed_python <=0:
    tokenized_python = tokenized_python[:50]
    tokenized_python.append(EOS_token)
    tokenized_python = torch.from_numpy(np.array(tokenized_python))
  else:
    tokenized_python = nn.functional.pad(torch.from_numpy(np.array(tokenized_python)), (0,padding_needed_python))
  final_prepared_data.append((tokenized_english.numpy(),tokenized_python.numpy()))

In [ ]:
training, testing = train_test_split(final_prepared_data, test_size=0.2,random_state=42,shuffle=True)

inputs_train = torch.Tensor([pair[0] for pair in training])
outputs_train = torch.Tensor([pair[1] for pair in training])
inputs_test = torch.Tensor([pair[0] for pair in testing])
outputs_test = torch.Tensor([pair[1] for pair in testing])

training_dataset = TensorDataset(inputs_train,outputs_train)
testing_dataset = TensorDataset(inputs_test,outputs_test)

training_dataloader = DataLoader(training_dataset,batch_size=32,shuffle=True)
testing_dataloader = DataLoader(testing_dataset,batch_size=32,shuffle=True)

<ipython-input-16-3cfb173caeb2>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  inputs_train = torch.Tensor([pair[0] for pair in training])


In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_size=english_lang.n_words, hidden_size=30, dropout_p=0.2):
    super().__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size,hidden_size)
    self.gru = nn.GRU(hidden_size,hidden_size,batch_first=True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self,inputs):
    embedded = self.dropout(self.embedding(inputs))
    outputs, hiddens = self.gru(embedded)
    return outputs, hiddens

In [ ]:
class Decoder(nn.Module):
  def __init__(self,hidden_size=30, output_size=python_lang.n_words):
    super().__init__()
    self.embedding = nn.Embedding(output_size,hidden_size)
    self.gru = nn.GRU(hidden_size,hidden_size,batch_first=True)
    self.out = nn.Linear(hidden_size,output_size)

    # ANN for attention
    self.attent1 = nn.Linear(hidden_size * 2,50)
    self.attention_out = nn.Linear(50,1)

  def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):#prediction model
    batch_size = encoder_outputs.size(0)
    decoder_input = torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
    decoder_hidden = encoder_hidden
    decoder_outputs = []

    for i in range(51):
      decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
      decoder_outputs.append(decoder_output)

      if target_tensor is not None:
        decoder_input = target_tensor[:,i].unsqueeze(1)
      else:
        _, topi = nn.functional.softmax(decoder_output,dim=1).topk(1)
        decoder_input = topi.squeeze(-1).detach()

    decoder_outputs = torch.cat(decoder_outputs, dim=1)
    return decoder_outputs, decoder_hidden
  def forward_step(self,inputs,hidden):
    output = self.embedding(inputs)
    output, hidden = self.gru(output, hidden)
    output = self.out(output)
    return output, hidden

In [ ]:
encoder = Encoder(hidden_size=200)
decoder = Decoder(hidden_size=200)#hidden size shows you how big the vector is
encoder.to(device)
decoder.to(device)

loss_fn = nn.CrossEntropyLoss(reduction='sum')
encoder_optim = optim.Adam(encoder.parameters(),lr=0.001)#adam is like gradient descent it's an optimizer
decoder_optim = optim.Adam(decoder.parameters(),lr=0.001)

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_fn, dataset):
  encoder.train()
  decoder.train()
  total_loss = 0
  for X, y in dataloader:
    X = X.to(device)
    y = y.to(device)

    encoder_outputs, encoder_hidden = encoder(X.int())
    decoder_outputs, _ = decoder(encoder_outputs, encoder_hidden)

    y = nn.functional.one_hot(y.to(torch.int64), num_classes=python_lang.n_words)

    loss = loss_fn(decoder_outputs.float(),y.float())
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    total_loss += loss
  return total_loss/len(dataset)



In [ ]:
training_losses = []
epochs = 10 # change epoch to see different result

for epoch in range(epochs):
  loss = train_epoch(training_dataloader,encoder,decoder,encoder_optim,decoder_optim,loss_fn,training_dataset)
  training_losses.append(loss.item())
  print(f'Epoch {epoch}')
  print(loss.item())
  print()

Epoch 0
141.2779541015625

Epoch 1
141.24771118164062

Epoch 2
140.71400451660156

Epoch 3
140.49801635742188

Epoch 4
140.4216766357422

Epoch 5
140.21310424804688

Epoch 6
140.14756774902344

Epoch 7
139.88946533203125

Epoch 8
140.15213012695312

Epoch 9
139.2677001953125

